# Imports


In [1]:
##!pip install dash
import pandas as pd
from dash import Dash, html, dcc
import plotly.express as px
from dash.dependencies import Input, Output

# Tratando o Dataframe

In [10]:
df = pd.read_csv('testeDados.csv')
df_municipios = pd.read_csv('municipios_se.csv')
campos_vazios = df.isnull().sum()
mediana_acessos = df['acessos'].median()
data = df.drop(columns=["id_municipio", "sigla_uf", "cnpj"])


In [11]:
df['empresa'].fillna('outra', inplace=True)
df['porte_empresa'].fillna('Pequeno Porte', inplace=True)
df['tecnologia'].fillna('outra', inplace=True)
df['transmissao'].fillna('outra', inplace=True)
df['velocidade'].fillna('512kbps a 2Mbps', inplace=True)
df['acessos'].fillna(mediana_acessos, inplace=True)
df['produto'].fillna('internet', inplace=True)
df = df.dropna()

In [18]:
anos = sorted(df['ano'].unique())
df_media_por_mes = df.groupby(['mes','ano'])['acessos'].mean().reset_index()

Sigla_UF = sorted(df['sigla_uf'].unique())

data_estado = df.groupby(by=["ano", "mes", "empresa", "porte_empresa", "tecnologia", "transmissao", "velocidade","id_municipio", "sigla_uf", "cnpj"], as_index=False).sum()

def get_historico_tecnologia(data, mes, ano1, ano2):
    historico_tecnologia = data.query(f'ano > 2009 and ano < 2023 and mes == {mes}')
    historico_tecnologia = historico_tecnologia.filter(items=['ano', 'tecnologia', 'acessos'])
    historico_tecnologia = historico_tecnologia.groupby(['ano', 'tecnologia'])['acessos'].sum()
    historico_tecnologia = historico_tecnologia.to_frame().reset_index()

    df_historico_tecnologia = pd.DataFrame()
    for ano in historico_tecnologia['ano'].sort_values().unique():
        df_historico_tecnologia = pd.concat([historico_tecnologia.query('ano == @ano').nlargest(3, 'acessos'), df_historico_tecnologia])

    df_historico_tecnologia = df_historico_tecnologia.sort_values(['ano'])
    df_historico_tecnologia.columns = ['Ano', 'Tecnologia', 'Total de acessos']
    df_historico_tecnologia = df_historico_tecnologia.sort_values(['Ano', 'Total de acessos'])

    return df_historico_tecnologia

regioes = {
    'Grande Aracaju': ['Aracaju', 'Barra dos Coqueiros', 'Nossa Senhora do Socorro', 'São Cristóvão', "Itaporanga d'Ajuda", 'Laranjeiras', 'Riachuelo', 'Maruim', 'Santo Amaro das Brotas'],
    'Leste': ['Capela', 'Japaratuba', 'Siriri', 'Pirambu', 'Rosário do Catete', 'Carmópolis', 'General Maynard', 'Divina Pastora', 'Santa Rosa de Lima'],
    'Baixo São Francisco': ['Canhoba', 'Amparo de São Francisco', 'Telha', 'Propriá', 'Cedro de São João', 'Malhada dos Bois', 'São Francisco', 'Muribeca', 'Santana do São Francisco', 'Neópolis', 'Japoatã', 'Pacatuba', 'Ilha das Flores', 'Brejo Grande'],
    'Médio Sertão': ['Itabi', 'Gracho Cardoso', 'Aquidabã', 'Feira Nova', 'Cumbe', 'Nossa Senhora das Dores'],
    'Agreste Central': ['Carira', 'Nossa Senhora Aparecida', 'São Miguel do Aleixo', 'Ribeirópolis', 'Frei Paulo', 'Pinhão', 'Pedra Mole', 'Macambira', 'São Domingos', 'Campo do Brito', 'Itabaiana', 'Moita Bonita', 'Malhador', 'Areia Branca'],
    'Alto Sertão': ['Canindé de São Francisco', 'Poço Redondo', 'Porto da Folha', 'Gararu', 'Nossa Senhora de Lourdes', 'Nossa Senhora da Glória', 'Monte Alegre de Sergipe'],
    'Centro Sul': ['Simão Dias', 'Poço Verde', 'Tobias Barreto', 'Riachão do Dantas', 'Lagarto'],
    'Sul': ['Salgado', 'Boquim', 'Estância', 'Pedrinhas', 'Arauá', 'Itabaianinha', 'Tomar do Geru', 'Cristinápolis', 'Umbaúba', 'Indiaroba', 'Santa Luzia do Itanhy']
}


In [17]:
app = Dash(__name__)


app.layout = html.Div(children=[
    html.H1(children='Análise dos Dados de Banda Larga', style={'font-family': 'Arial, sans-serif', 'text-align': 'center'}),
    html.Div(children='''
    1- Como está distribuído o número de acessos de acordo com os atributos apresentados?
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '30px','font-size':'20px','font-weight':'bold'}),
    dcc.Dropdown(
        id='ano-dropdown',
        options=[{'label': ano, 'value': ano} for ano in anos], 
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),
        html.Div(children=''' ''', style={'padding-bottom': '30px'}),

    dcc.Graph(
        id='graph-content'
    ),
    html.Div(children='''
    2- Quais as características em relação a tecnologia, transmissão e velocidade que predomina hoje em dia no escopo escolhido pelo grupo?
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '30px','font-size':'20px','font-weight':'bold'}),
    
    html.Div(children='''
    Ano:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px' , 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-Ano-dropdown',
        options=[{'label': ano, 'value': ano} for ano in anos], 
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),

    html.Div(children='''
    Mês:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px', 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-MES-dropdown',
        options=[{'label': mes, 'value': mes} for mes in range(1, 13)],
        value=1,
        style={'font-family': 'Arial, sans-serif'}
    ),
    dcc.Graph(
        id='graph-caracteristicas'
    ),
    dcc.Graph(
        id='graph-transmissao'
    ),
    dcc.Graph(
        id='graph-velocidade'
    ),

    html.Div(children='''
    3- Como essas características mudaram ao longo do ano?
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '30px','font-size':'20px','font-weight':'bold'}),

    html.Div(children='''
    Ano:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px' , 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-Ano-dropdown-2',
        options=[{'label': ano, 'value': ano} for ano in anos], 
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),

        html.Div(children='''
    Até:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px' , 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-Ano-dropdown-3',
        options=[{'label': ano, 'value': ano} for ano in anos], 
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),

    html.Div(children='''
    Mês:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px', 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-MES-dropdown-2',
        options=[{'label': mes, 'value': mes} for mes in range(1, 13)],
        value=1,
        style={'font-family': 'Arial, sans-serif'}
    ),

    dcc.Graph(
        id='graph-historico-tecnologia'
    ),

    html.Div(children='''
    4- Quais empresas com maior número de contratos considerando o porte da empresa?
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '30px','font-size':'20px','font-weight':'bold'}),

    html.Div(children='''
    Ano:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px' , 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-Ano-dropdown-4',
        options=[{'label': ano, 'value': ano} for ano in anos], 
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),


    html.Div(children='''
    Mês:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px', 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-MES-dropdown-3',
        options=[{'label': mes, 'value': mes} for mes in range(1, 13)],
        value=1,
        style={'font-family': 'Arial, sans-serif'}
    ),

    
    html.Div(children='''
    Porte:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px', 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-porte-dropdown',
        options=['Grande Porte','Pequeno Porte'],
        value="Grande Porte",
        style={'font-family': 'Arial, sans-serif'}
    ),


    dcc.Graph(
        id='graph-porte'
    ),

    html.Div(children='''
    5- Como o perfil do tipo de empresa mudou ao longo dos anos nos estados? Houve crescimento ou redução em relação a um tipo de porte específico?
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '30px','font-size':'20px','font-weight':'bold'}),

    html.Div(children='''
    Ano:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px' , 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-Ano-dropdown-5',
        options=[{'label': ano, 'value': ano} for ano in anos], 
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),

    html.Div(children='''
    Até:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px' , 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-Ano-dropdown-6',
        options=[{'label': ano, 'value': ano} for ano in anos], 
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),

    html.Div(children='''
    Mês:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px', 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-MES-dropdown-4',
        options=[{'label': mes, 'value': mes} for mes in range(1, 13)],
        value=1,
        style={'font-family': 'Arial, sans-serif'}
    ),


    dcc.Graph(
        id='graph-perfil'
    ),

    html.Div(children='''
    6- Atualmente, usando os dados mais recentes da base, qual o perfil da distribuição de banda larga dentro do escopo escolhido pelo grupo? escolha escopos que permitam a análise a partir de 2 ou mais parâmetros.
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '30px','font-size':'20px','font-weight':'bold'}),

    html.Div(children='''
    Ano:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px' , 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-Ano-dropdown-7',
        options=[{'label': ano, 'value': ano} for ano in anos], 
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),
    html.Div(children='''
    Mês:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px', 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-MES-dropdown-5',
        options=[{'label': mes, 'value': mes} for mes in range(1, 13)],
        value=1,
        style={'font-family': 'Arial, sans-serif'}
    ),

    dcc.Graph(
        id='graph-porte-velocidade'
    ),

])

@app.callback(
    Output('graph-content', 'figure'),
    [Input('ano-dropdown', 'value')]
)

def update_graph(selected_ano):
    df_ano_selecionado = df_media_por_mes[df_media_por_mes['ano'] == selected_ano]
    fig_ano_mes = px.bar(df_ano_selecionado, x='mes', y='acessos', title=f'Média de Acessos por Mês no Ano de: {selected_ano}', barmode='group')
    
    return fig_ano_mes

# Callback para atualizar o gráfico
@app.callback(
    Output('graph-caracteristicas', 'figure'),
    [Input('caracteristica-Ano-dropdown', 'value'),
     Input('caracteristica-MES-dropdown', 'value'),]
)
def update_graph_2(ano, mes):
    analise_tecnologias_recentes = df.query(f'ano == {ano} and mes == {mes}')
    analise_tecnologias_recentes = analise_tecnologias_recentes.groupby('tecnologia')['acessos'].sum()
    analise_tecnologias_recentes = pd.DataFrame({'Tecnologia': analise_tecnologias_recentes.index, 'Total de acessos': analise_tecnologias_recentes.values})
    analise_tecnologias_recentes = analise_tecnologias_recentes.sort_values(['Total de acessos'], ascending=False)

    fig = px.bar(analise_tecnologias_recentes, x='Tecnologia', y='Total de acessos', title='Análise de Tecnologias Recentes')
    return fig
# Callback para atualizar o gráfico
@app.callback(
    Output('graph-transmissao', 'figure'),
    [Input('caracteristica-Ano-dropdown', 'value'),
     Input('caracteristica-MES-dropdown', 'value'),]
)
def update_graph_3(ano, mes):
    analise_transmissao_recentes = data_estado.filter(items=['ano', 'mes', 'transmissao', 'acessos'])
    analise_transmissao_recentes = analise_transmissao_recentes.query(f'ano == {ano} and mes == {mes}')
    analise_transmissao_recentes = analise_transmissao_recentes.groupby('transmissao')['acessos'].sum()
    analise_transmissao_recentes = pd.DataFrame({'Transmissão': analise_transmissao_recentes.index, 'Total de acessos': analise_transmissao_recentes.values})
    analise_transmissao_recentes = analise_transmissao_recentes.sort_values(['Total de acessos'], ascending=False)

    fig = px.bar(analise_transmissao_recentes, x='Transmissão', y='Total de acessos', title='Análise de Transmissão Recentes')
    return fig

@app.callback(
    Output('graph-velocidade', 'figure'),
    [Input('caracteristica-Ano-dropdown', 'value'),
     Input('caracteristica-MES-dropdown', 'value'),]
)
def update_graph_4(ano, mes):
    analise_velocidade_recentes = data_estado.filter(items=['ano', 'mes', 'velocidade', 'acessos'])
    analise_velocidade_recentes = analise_velocidade_recentes.query(f'ano == {ano} and mes == {mes}')
    analise_velocidade_recentes = analise_velocidade_recentes.groupby('velocidade')['acessos'].sum()
    analise_velocidade_recentes = pd.DataFrame({'Velocidade': analise_velocidade_recentes.index, 'Total de acessos': analise_velocidade_recentes.values})
    analise_velocidade_recentes = analise_velocidade_recentes.sort_values(['Total de acessos'], ascending=False)

    fig = px.bar(analise_velocidade_recentes, x='Velocidade', y='Total de acessos', title='Análise de Velocidade Recentes')
    return fig

@app.callback(
    Output('graph-historico-tecnologia', 'figure'),
    [Input('caracteristica-Ano-dropdown-2', 'value'),
     Input('caracteristica-Ano-dropdown-3', 'value'),
     Input('caracteristica-MES-dropdown-2', 'value')]
)
def update_graph_5(ano2,ano3, mes):
    df_historico_tecnologia = get_historico_tecnologia(df,mes,ano2, ano3)

    # Filtre o DataFrame com base no ano e mês selecionados
    df_historico_tecnologia_filtrado = df_historico_tecnologia[(df_historico_tecnologia['Ano'] == ano2)]

    # Crie o gráfico de barras
    fig = px.bar(df_historico_tecnologia_filtrado, x='Tecnologia', y='Total de acessos', title=f'Histórico de Tecnologia para o Ano {ano2} até o ano {ano3} e Mês {mes}')
    return fig

@app.callback(
    Output('graph-porte', 'figure'),
    [Input('caracteristica-Ano-dropdown-4', 'value'),
     Input('caracteristica-MES-dropdown-3', 'value'),
     Input('caracteristica-porte-dropdown', 'value')]
)
def update_graph_6(ano,mes,porte):
    # Sua lógica original para obter os dados
    analise_empresa = df.query(f'ano == {ano} and mes == {mes} and porte_empresa == "{porte}"')
    analise_empresa = analise_empresa.groupby(['empresa', 'porte_empresa'])['acessos'].sum()
    analise_empresa = analise_empresa.to_frame().reset_index()
    analise_empresa = analise_empresa.sort_values('acessos', ascending=False)

    # Agora, crie o gráfico com base nos dados processados
    fig = px.bar(analise_empresa, x='empresa', y='acessos', color='porte_empresa', barmode='group')

    return fig

@app.callback(
    Output('graph-perfil', 'figure'),
    [Input('caracteristica-Ano-dropdown-5', 'value'),
     Input('caracteristica-Ano-dropdown-6', 'value'),
     Input('caracteristica-MES-dropdown-4', 'value'),]
)
def update_graph_7(ano1,ano2,mes):
    # Sua lógica original para obter os dados
    historico_acessos_porte = data_estado.query(f'mes == {mes} and ano > {ano1} and ano <= {ano2}')
    historico_acessos_porte = historico_acessos_porte.filter(['ano', 'mes', 'porte_empresa', 'acessos'])
    historico_acessos_porte = historico_acessos_porte.groupby(['ano', 'mes', 'porte_empresa'])['acessos'].sum()
    historico_acessos_porte = historico_acessos_porte.to_frame()
    historico_acessos_porte = historico_acessos_porte.reset_index()
    historico_acessos_porte.columns= ['Ano', 'Mês', 'Porte empresa', 'Total de acessos']

    # Agora, crie o gráfico com base nos dados processados
    fig = px.line(historico_acessos_porte, x="Ano", y="Total de acessos", color='Porte empresa', title=f'Comparação histórica do numero de acessos por porte da empresa no mês {mes}', markers=True, color_discrete_sequence=["#24389e", "#23979b"])

    return fig

@app.callback( 
    Output('graph-porte-velocidade', 'figure'),
    [Input('caracteristica-Ano-dropdown-7', 'value'),
     Input('caracteristica-MES-dropdown-5', 'value'),]
)
def update_graph_8(ano,mes):
    # Sua lógica original para obter os dados
    analise_velocidade_porte = df.query(f'ano == {ano} and mes == {mes}')
    analise_velocidade_porte = analise_velocidade_porte.groupby(['velocidade', 'porte_empresa'])['acessos'].sum()
    analise_velocidade_porte = analise_velocidade_porte.to_frame().reset_index()

    # Agora, crie o gráfico com base nos dados processados
    fig = px.bar(analise_velocidade_porte, x='velocidade', y='acessos', color='porte_empresa', barmode='group')

    return fig


if __name__ == '__main__':
    app.run_server(debug=True)